# Importing packages 

In [1]:
# Data wrangling 
import pandas as pd 

# Array math
import numpy as np 

# Ploting 
import seaborn as sns
import matplotlib.pyplot as plt

# List iteration tracking
from tqdm import tqdm

# Importing the custom written class 
from DecisionTree import Node 

# Importing the custom regression tree 
from RandomForest import RandomForestClassifier, RandomForestTree

# Time tracking
import time

# Reading data 

The data regards telecom churn. 

The objective is to create a model that predicts whether a customer will quit using the features available.

In [2]:
d = pd.read_csv('data/random_forest/telecom_churn.csv')

In [3]:
print(f"Data shape: {d.shape}")

Data shape: (3333, 11)


In [4]:
d.head()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.7,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.7,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.0,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.0,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.0,3,166.7,113,41.0,7.42,10.1


In [5]:
d.dtypes

Churn                int64
AccountWeeks         int64
ContractRenewal      int64
DataPlan             int64
DataUsage          float64
CustServCalls        int64
DayMins            float64
DayCalls             int64
MonthlyCharge      float64
OverageFee         float64
RoamMins           float64
dtype: object

In [6]:
# Distribution of churn in data 
d.groupby('Churn').size()

Churn
0    2850
1     483
dtype: int64

# Random forest - quick theory review

The classifier which will be created is a random forest classifier. 

Lets denote it as **rf()**.  

Given a set of input matrix $\mathbb{X}_{nxp}$ the classifier **rf()** outputs either 1 or 0.

$$rf: \mathbb{X} \rightarrow \{1, 0\}$$

The algorithm of the random forest grows **k** decision trees. 

The final prediction of the **rf()** classifier is a majority vote: the input matrix $\mathbb{X}$ is used with each of the **k** trees, and then the class with the most outputs wins. 

In the notebook about decision trees it is clear that with the same input and the same hyperparameters, the same output and the same rules will be learnt by a decision tree. So why grow **k** of them? 

## Data bootstrapping

The random in the random forest starts at the data sample creation for each of the decision trees. The technique used in creating **k** datasamples is bootstrapping

Given a dataset of n rows and p features: we sample the rows from the original dataset with replacement. For every new decision tree *i*, a new bootsrapped dataset is created: $\mathbb{X_{b}^{i}}$.

For example, lets assume that the whole dataset has 10 rows of data:

In [16]:
# Lets imagine this the whole dataset
dsubset = d.sample(10).copy()[['Churn', 'DataPlan', 'DayMins', 'OverageFee']]
dsubset.reset_index(inplace=True, drop=True)

print(dsubset)

   Churn  DataPlan  DayMins  OverageFee
0      0         0    179.2       11.15
1      0         0    228.4        7.26
2      1         0    286.2        9.36
3      0         1    198.8       11.93
4      1         0    312.0        6.47
5      0         0     90.6        8.53
6      0         0    220.9        9.61
7      0         0    203.2        7.63
8      0         0    111.1       15.67
9      0         0    100.8       11.50


To create 3 more random bootsrapped samples we use the pandas function **sample(replace=True)**. The key concept is that the sampling is done *with replacement*: the same rows might appear several times in our sample. 

In [19]:
for i, _ in enumerate(range(3)):
    print("----- \n")
    print(f"Boostrapped sample: {i + 1} \n")
    print(dsubset.sample(frac=1.0, replace=True))
    print("----- \n")

----- 

Boostrapped sample: 1 

   Churn  DataPlan  DayMins  OverageFee
7      0         0    203.2        7.63
2      1         0    286.2        9.36
2      1         0    286.2        9.36
8      0         0    111.1       15.67
7      0         0    203.2        7.63
1      0         0    228.4        7.26
6      0         0    220.9        9.61
9      0         0    100.8       11.50
3      0         1    198.8       11.93
6      0         0    220.9        9.61
----- 

----- 

Boostrapped sample: 2 

   Churn  DataPlan  DayMins  OverageFee
7      0         0    203.2        7.63
3      0         1    198.8       11.93
7      0         0    203.2        7.63
5      0         0     90.6        8.53
9      0         0    100.8       11.50
0      0         0    179.2       11.15
9      0         0    100.8       11.50
7      0         0    203.2        7.63
8      0         0    111.1       15.67
5      0         0     90.6        8.53
----- 

----- 

Boostrapped sample: 3 

   Churn

For each of the **k** trees grown in random forest, we create **k** bootstrapped data samples. 

## Feature selection at each split 

Now that we have a dataset $\mathbb{X_{b}^{i}}$ for each of the **k** trees the final part is to determine the splitting criterion for the creation of the nodes. 

In the classification case, the gini gain criterion is the same as in the simple decision tree case. The difference is that at each node splitting, a random subsample of collumns are select to find the "best split". 

For example, if we have 10 collumns as features and we select the hyperparameter of **X_features_fraction = 0.8** then at each node where the best split is beeing calculated, we would select 8 random features (10 * 0.8 = 8).  

# Features to use 

The bellow feature list will be used in the creation of the random forest. 

In [21]:
# Defining the feature list used in the growth of the tree
features = [
    'AccountWeeks',
    'DataUsage',
    'DayMins',
    'DayCalls',
    'MonthlyCharge',
    'OverageFee',
    'RoamMins'
]

# Creating the train and test sets 

In [23]:
# Fraction of rows in the training set 
train_share = 0.75

# Creating the train and test sets
train = d.sample(frac=train_share)
test = d[~d.index.isin(train.index)]

print(f"Total rows in the dataset: {d.shape[0]}")
print(f"Rows in training set: {train.shape[0]}")
print(f"Rows in test set: {test.shape[0]}")

Total rows in the dataset: 3333
Rows in training set: 2500
Rows in test set: 833


# Training the random forest 

In [24]:
# Initiating the random forest object 
rf = RandomForestClassifier(
    Y=train['Churn'], 
    X=train[features],
    min_samples_split=10,
    max_depth=3,
    n_trees=10, # Number of trees grown
    X_features_fraction=0.75,
    X_obs_fraction=0.75
    )

# Growing the random forest 
rf.grow_random_forest()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:44<00:00,  4.44s/it]
